In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sys
import json
import pickle
import json

from tqdm import tqdm
import itertools

from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_recall_curve, roc_curve, auc
from sklearn.metrics import make_scorer, roc_auc_score

from torchsurv.loss import cox
from lifelines.utils import concordance_index

sys.path.append('./../src/')
from utils import *
from utils_deepsurv import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset


# Configs

In [2]:
# create config files
lr_ls = [0.1, 0.001, 0.00001]
hidden_size_ls = [64, 128, 256]
l2_reg_ls = [0.1, 0.01, 0.001]
batch_size = [32, 64]
dropout_ls = [0, 0.1, 0.2]


In [3]:
for idx, (lr_, hidden_size_, l2_reg_, batch_size_, dropout_) in tqdm(enumerate(itertools.product(lr_ls, hidden_size_ls, l2_reg_ls, batch_size, dropout_ls))):
    with open('./DeepSurv/configs/config_'+str(idx)+'.json', 'w') as f:
        json.dump({'idx':idx,
                     'lr':lr_,
                     'hidden_size':hidden_size_,
                     'l2_reg':l2_reg_,
                     'batch_size':batch_size_,
                     'dropout':dropout_}, f)
    f.close()

162it [00:00, 1708.44it/s]


# Hyperparameter tuning result

In [16]:
lr_ls = [0.1, 0.001, 0.00001]
hidden_size_ls = [64, 128, 256]
l2_reg_ls = [0.1, 0.01, 0.001]
batch_size = [32, 64]
dropout_ls = [0, 0.1, 0.2]

combinations = [i for i in itertools.product(lr_ls, hidden_size_ls, l2_reg_ls, batch_size, dropout_ls)]
ncombinations = len([i for i in itertools.product(lr_ls, hidden_size_ls, l2_reg_ls, batch_size, dropout_ls)])
ncombinations


162

In [20]:
results_ls = []
for i in range(ncombinations):
    with open('./DeepSurv/results/result_config_'+str(i)+'.pkl','rb') as f:
        result = pickle.load(f)
    f.close()
    df = pd.DataFrame(result)
    df['idx'] = [i]*3
    results_ls = results_ls + [df.mean().to_frame().T]
    
pd.concat(results_ls).sort_values('c_index_valid')

,seed,learning_rate,hidden_size,l2_reg,batch_size,dropout,time,c_index_train,c_index_valid,c_index_test,nepochs,idx
0,349.333333,0.10000,128.0,0.100,32.0,0.2,55.055543,0.485749,0.409130,0.461804,93.666667,20.0
0,349.333333,0.10000,256.0,0.100,32.0,0.1,147.984298,0.511771,0.480408,0.522819,156.333333,37.0
0,349.333333,0.10000,64.0,0.010,32.0,0.0,27.011605,0.590738,0.492432,0.609223,95.000000,6.0
0,349.333333,0.10000,256.0,0.010,64.0,0.1,16.325075,0.475562,0.496029,0.486097,57.000000,46.0
0,349.333333,0.10000,64.0,0.100,32.0,0.0,34.322748,0.500000,0.500000,0.500000,105.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,349.333333,0.00001,64.0,0.010,64.0,0.1,34.720910,0.914962,0.751818,0.739970,162.333333,118.0
0,349.333333,0.00001,64.0,0.010,64.0,0.2,42.813808,0.901809,0.754201,0.742462,162.333333,119.0
0,349.333333,0.00001,128.0,0.010,64.0,0.2,47.572840,0.926558,0.755618,0.758292,170.333333,137.0
0,349.333333,0.00001,64.0,0.010,64.0,0.0,29.880593,0.923724,0.761283,0.750223,148.000000,117.0
